<a href="https://colab.research.google.com/github/Leeuenho/image_deep_learning/blob/main/04_NN_backprop_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 1. 지난시간 복습

Cross-entrophy loss는 다음과 같이 정의한다. 

$CELoss = -  \sum^C_{i=1} (y_i \cdot \log \hat{y_i})$


여기서 C는 분류할 클래스의 개수이다. 
우리는 Loss를 모든 데이터에 기반하여 계산할 것이므로, 최종으로 구할 L는 모든 데이터 M개에 관한 CELoss의 평균을 이용한다. 

$L = - \sum^{M}_{j=1} \sum^C_{i=1} (y_i^j \cdot \log \hat{y_i^j})$

일반적으로 CELoss는 Softmax가 적용된 출력을 입력으로 받으며, softmax+CELoss를 입력값으로 미분하게 될 때 매우 간결한 형태의 analytic gradient를 얻는다. 

- $\frac{\partial L}{\partial z_i} = p_i - y_i$


- 여기서 $z_i$는 softmax에 들어가는 입력 (혹은 앞 레이어의 출력)



CELoss + Softmax 결합 모듈의 미분 증명 

- https://levelup.gitconnected.com/killer-combo-softmax-and-cross-entropy-5907442f60ba


In [1]:
import numpy as np

# softmax function: np.exp에 너무 큰값이 들어가지 않도록 수정
def Softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# CELoss 
def CELoss(Y_hat, labels): 
    n_data = len(labels)
    loss = 0 
    for i in range(n_data):        
        celoss = np.sum(- (labels[i]*np.log(Y_hat[i])))
        loss += celoss/n_data
    return loss
    
# 데이터
x_1 = np.array([[56, 231],
                   [24, 1]])
x_2 = np.array([[120, 30],
                   [24, 0]])
x_3 = np.array([[20, 31],
                   [240, 100]])
x_4 = np.array([[56, 201],
                   [22, -10]])
x_5 = np.array([[140, 27],
                   [30, 10]])
x_6 = np.array([[25, 30],
                   [230, 110]])


# one-hot vector 정의하기
y_1 = np.array([1,0,0])   #label_cat
y_2 = np.array([0,1,0])   #label_dog
y_3 = np.array([0,0,1])   #label_ship 
y_4 = np.array([1,0,0])   #label_cat_2
y_5 = np.array([0,1,0])   #label_dog_2
y_6 = np.array([0,0,1])   #label_ship_2 


#  데이터셋
X = np.array([x_1, x_2, x_3, x_4, x_5, x_6]) # data
Y = np.array([y_1, y_2, y_3, y_4, y_5, y_6]) # label

print(X.shape)
print(Y.shape)

(6, 2, 2)
(6, 3)


## 실습과제 . 지난시간에 구현한 뉴럴넷 Class 형태로 재구현

지난 시간에 구현하였던 간단한 선형분류기+Softmax+CELoss 구조를 갖는 뉴럴네트워크를 아래와 같이 python class로 구현하였다. 

- 구조: x -> Linear(4,3) -> Softmax -> CELoss <- y_hat

여기서 Linear(in, out)은 입력in개와 출력 out개를 갖는 선형분류기를 뜻한다. Backward 함수를 구현하여 본 뉴럴넷의 학습을 동작하게 만드시오. 각 레이어에 대한 local gradient 계산법은 아래에 첨부하였다. 

------------------------------

각 레이어에 대한 미분값

Derivative of CELoss with softmax :

- $\frac{\partial L}{\partial x_i} = p_i - y_i$

- 여기서 $x_i$는 softmax에 들어가는 입력 (혹은 앞 레이어의 출력 $z_i$)

- ($L = CELoss(softmax(x), y) = CELoss(p,y)$) 

- https://levelup.gitconnected.com/killer-combo-softmax-and-cross-entropy-5907442f60ba

Derivative of Linear layer (Y = WX):

- $\frac{\partial L}{\partial W} = \frac{\partial L}{\partial Y} X^T$

- $\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W^T$


- http://cs231n.stanford.edu/handouts/linear-backprop.pdf





In [4]:
# Single-layer Neural Network class

class NN_1layer():
    def __init__(self, **kwargs):
        self.W_l1 = np.random.rand(3,4) #가중치
        self.activation = Softmax
        self.Loss = CELoss
        self.learning_rate = 0.0001

    def Forward(self, X):
        # initial prediction
        Y_hat = []
        
        for x in X: 
            # layer 1
            x = np.matmul(self.W_l1, x.flatten())
          
            # softmax
            p = self.activation(x)

            Y_hat.append(p)
        return Y_hat
        
    def getLoss(self, X, Y):
        # y_hat -> CELoss <- y
        Y_hat = self.Forward(X)
        return self.Loss(Y_hat, Y)

    def Backward(self, Y_hat, X, Y): 
        dW1Loss = np.zeros_like(self.W_l1)

        ####################################
        ##### 이곳에 코드를 작성하시오 #####
        # backpropagation 구현 힌트 
        # 1. softmax와 CELoss는 한개의 레이어로 간주하고 미분
        #   x -> [Linear(4,3)] -> [Softmax -> CELoss] <- y_hat
        # 2. 두 레이어의 gradient 공식은 위의 각 레이어 미분 공식을 분석
        #   분석 시 함수, 입력, 출력 기호를 명확하게 파악할 것!
        
        # Backpropagation 공식
        #   Downstream gradient = UpstreamGradient * LocalGradient
        # - 각 레이어 별 LocalGradient 계산법을 파악
        # - 각 레이어 별 UpstreamGradient가 무엇인지 파악
        # - 위 두가지를 곱하여 downstreamGradient를 계산
      
        ##### 참고 사항. np 행렬의 형상 차이에 주의 
        # shape (N,)은 1차원 배열로써 행렬의 transpose가 적용되지 않으므로 
        # reshape(N,1)로 차원을 명시한 뒤에 transpose를 적용하시오. 
        # 사용 예)  x.reshape(4,1).T


        ####################################
        
        up = Y_hat - Y
        local = X.reshape(-1,4)
        
        z = np.matmul(up.T, local)
        
        dW1Loss = z
        return dW1Loss



    def training(self, X, Y, n_epoch=100):
        iterations = 0
        while iterations < n_epoch: 
            # gradient descent algorithm 
            Y_hat = self.Forward(X)
            dWLoss = self.Backward(Y_hat, X, Y)

            self.W_l1 = self.W_l1 - dWLoss*self.learning_rate
            print(f'iteration: {iterations+1}, loss: {self.getLoss(X,Y)}')
            iterations += 1



my_NeuralNet = NN_1layer()
my_NeuralNet.training(X, Y)

# 학습이 잘되었다면 [1,0,0] 과 같은 형태로 출력
#print(my_NeuralNet.Forward(X))

# 학습이 잘되었다면 매우 작은값(예: 0.01) 형태가 출력된다.
#print(my_NeuralNet.getLoss(X, Y))


iteration: 1, loss: 55.125733648070806
iteration: 2, loss: 40.86874744798082
iteration: 3, loss: 26.749662212881145
iteration: 4, loss: 21.661776862935756
iteration: 5, loss: 19.761202293128406
iteration: 6, loss: 18.23692714735504
iteration: 7, loss: 16.74961654727891
iteration: 8, loss: 15.207849807881935
iteration: 9, loss: 13.807264653451103
iteration: 10, loss: 12.253901554774897
iteration: 11, loss: 10.85228131008338
iteration: 12, loss: 9.31667379007588
iteration: 13, loss: 7.916788542941438
iteration: 14, loss: 6.369900939955491
iteration: 15, loss: 4.969441918977458
iteration: 16, loss: 3.4296318958373515
iteration: 17, loss: 2.03211542895123
iteration: 18, loss: 0.7011573011061116
iteration: 19, loss: 0.23654702936555894
iteration: 20, loss: 0.052075476650793144
iteration: 21, loss: 0.03781211391007523
iteration: 22, loss: 0.029812097635746975
iteration: 23, loss: 0.024657924461979307
iteration: 24, loss: 0.021049204814997776
iteration: 25, loss: 0.018376544072051956
iteratio

-----------------------------------------

# 심화 연습문제 (optional)

위 네트워크 구조를 아래와 같이 2 layer로 바꾸어 보고, 학습을 성공시켜 보시오. init(), Forward와 Backward 함수를 수정해야 한다. 

- 구조: x -> Linear(4,4) -> ReLU ->  Linear(4,3) -> Softmax -> CELoss <- y_hat

ReLU는 아래의 구현을 참고하시오. 


In [ ]:
# ReLU
def ReLU(x):
    return np.maximum(0, x)

# derivatives of ReLU
def dReLU(x):
    # if (x > 0):
    #     return 1
    # if (x <= 0):
    #     return 0
    return 1 * (x > 0)


In [ ]:
####################################
##### 이곳에 코드를 작성하시오 #####





####################################